In [1]:
pwd

'/data2/mito_lineage/src/vireo'

In [2]:

INDIR="/data/isshamie/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC5_hetCount2_bq20/mgatk/vireoIn/multiplex"
OUTDIR = "/data/isshamie/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC5_hetCount2_bq20/mgatk/vireoIn/multiplex/dendrograms"
sample_names =  "Control,Flt3l"
N_DONORS=4
#curr_dir="/data2/mito_lineage"
# INDIR="data/processed/mttrace/CHIP_april08_2021/MTblacklist/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/multiplex"
# OUTDIR="data/processed/mttrace/CHIP_april08_2021/MTblacklist/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/filter_mgatk/vireoIn/dendrograms"
# #OUTDIR= "data/CHIP_april08_2021/MTblacklist/chrM/pseudo/minC200_minAF0.01/numC100000_ispropFalse/variants"
# N_DONORS=5 
# sample_names="Control,Flt3l,Input"

In [3]:
import os
from src.utils.data_io import setup_outdir
#os.chdir(curr_dir)
setup_outdir(OUTDIR, dirs=('figures',))

In [4]:
sample_names = {ind:val for ind,val in enumerate(sample_names.split(","))}

In [5]:
import mplh.cluster_help as ch
from os.path import join, exists, dirname
import matplotlib.pyplot as plt
import vireoSNP
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.io import mmread

In [6]:
def filt_high(df, thresh):
    return df.loc[(df.mean(axis=1)<thresh)].index.values


# Using vireo for clonal reconstruction - mitochondrial mutations

The mitochondrial mutations data set is extracted from [Ludwig et al, Cell, 2019](https://doi.org/10.1016/j.cell.2019.01.022), the 9 variants used here are from Supp Fig. 2F (and main Fig. 2F).

For clonal reconstruction, we are using the core class `vireoSNP.Vireo` with multiple initializations

## Load cluster results and test for enrichment

In [7]:
import seaborn as sns
from mplh import cluster_help as ch

In [8]:
cells_meta = pd.read_csv(join(INDIR, "cells_meta.tsv"), index_col=0, sep='\t')
cells_meta = cells_meta.astype({"donor":"Int64", 'donor_index': 'Int64'})
cells_meta

,ID,raw ID,new index,condition,donor,donor_index
index,,,,,,
0,AAACGAAAGAGGTCCA-1_Control,AAACGAAAGAGGTCCA-1,1,Control,0,1
1,AAACGAAAGCGATACG-1_Control,AAACGAAAGCGATACG-1,2,Control,2,1
2,AAACGAAAGTCGTGAG-1_Control,AAACGAAAGTCGTGAG-1,3,Control,3,1
3,AAACGAACAATAGTGA-1_Control,AAACGAACAATAGTGA-1,4,Control,0,2
4,AAACGAACACAATAAG-1_Control,AAACGAACACAATAAG-1,5,Control,3,2
...,...,...,...,...,...,...
17400,TTTGTGTTCGAGTTAC-1_Flt3l,TTTGTGTTCGAGTTAC-1,17401,Flt3l,3,4204
17401,TTTGTGTTCGCATAAC-1_Flt3l,TTTGTGTTCGCATAAC-1,17402,Flt3l,<NA>,<NA>
17402,TTTGTGTTCGTGGTAT-1_Flt3l,TTTGTGTTCGTGGTAT-1,17403,Flt3l,2,3853


In [ ]:
#n_clone_list = [7]#[2,3,4,5,6,7]

for n in range(N_DONORS):
    out_f = join(OUTDIR, 'figures',  f"donor{n}_dendrogram.png")
    curr_ad_f = join(INDIR, f"donor{n}.AD.mtx")
    curr_dp_f = join(INDIR, f"donor{n}.DP.mtx")
    print(curr_ad_f)
    print(curr_dp_f)
    curr_ad = mmread(curr_ad_f).tocsc()
    curr_dp = mmread(curr_dp_f).tocsc()
    AD_df = pd.DataFrame(curr_ad.todense())
    DP_df = pd.DataFrame(curr_dp.todense())
    DP_df.index.name = "Variant"
    DP_df.columns.name = "Cell"
    AD_df.index.name = "Variant"
    AD_df.columns.name = "Cell"
    AF_df = AD_df/(DP_df+0.001)
    #AD_df = pd.merge(AD_df, vcf[["#CHROM", "POS", "ALT"]], how="inner", left_index=True,right_index=True).set_index(["#CHROM", "POS", "ALT"])
    curr_labels = cells_meta.loc[cells_meta['donor']==n].copy().sort_values("donor_index").reset_index()    
    
    if (AF_df.shape[1] > 500):
        cols=np.random.choice(AF_df.columns,size=500, replace=False)
    else:
        cols = AF_df.columns
    if (AF_df.shape[0] > 500):
        rows = AF_df.mean(axis=1).sort_values(ascending=False).index[:500]
    else:
        rows = AF_df.index
    
    nvars = AF_df.shape[0]
    ncells = AF_df.shape[1]
    
    AF_df = AF_df.loc[rows,cols]
    DP_df = DP_df.loc[rows,cols]
    curr_labels = curr_labels.loc[cols]
    
    g = ch.plot_cluster(np.sqrt(AF_df), col_meta=curr_labels[["condition"]], to_row_clust=True, to_col_clust=True, use_white=False )
    #g = sns.clustermap(np.sqrt(AF_df), col_colors=curr_labels[["Color"]], to_row_clust=True, to_col_clust=True, metric='jaccard')
    plt.suptitle(f"Donor:{n} Square root variant allele frequency\nvars:{nvars} cells:{ncells}")
    plt.savefig(out_f.replace(".png",".withHigh.png"))
    
    varsLow = filt_high(AF_df, thresh=0.9)
    AF_df = AF_df.loc[varsLow]
    DP_df = DP_df.loc[varsLow]
    
    g = ch.plot_cluster(np.sqrt(AF_df), col_meta=curr_labels[["condition"]], to_row_clust=True, to_col_clust=True, use_white=False )
    
    plt.suptitle(f"Donor:{n} Square root variant allele frequency removing donor-specific variants\nvars:{nvars} cells:{ncells}")
    plt.savefig(out_f)
    
    inds = g.dendrogram_row.dendrogram["leaves"]
    cols = g.dendrogram_col.dendrogram["leaves"]

    ch.plot_cluster(np.log2(DP_df.iloc[inds, cols]+1), to_row_clust=False, to_col_clust=False, 
                            col_meta=curr_labels.iloc[cols][["condition"]], use_white=False)

    plt.suptitle(f"Donor:{n}\nLog2 counts")
    plt.savefig(out_f.replace(".png",".depth.png"))

/data/isshamie/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC5_hetCount2_bq20/mgatk/vireoIn/multiplex/donor0.AD.mtx
/data/isshamie/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC5_hetCount2_bq20/mgatk/vireoIn/multiplex/donor0.DP.mtx
clustering
cmap None
clustering
cmap None
clustering
cmap None
/data/isshamie/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC5_hetCount2_bq20/mgatk/vireoIn/multiplex/donor1.AD.mtx
/data/isshamie/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC5_hetCount2_bq20/mgatk/vireoIn/multiplex/donor1.DP.mtx
clustering
cmap None
clustering
cmap None


In [10]:
# import mplh.cluster_help as ch

# ch.plot_cluster(np.log2(DP_df.iloc[inds, cols2]+1), to_row_clust=False, to_col_clust=False, 
#                         col_meta=curr_labels.iloc[cols2][["condition"]], use_white=False )
